In [1]:
import os
import pandas as pd
import re
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast, GradScaler
import torch.nn.functional as F
import numpy as np
import time

In [2]:

def ReadTxtIntoDataset(folder_path):
    folder_path = folder_path
    # 获取文件夹中所有文件的列表
    files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
    x_list = []
    # 读取每个文件中的数据
    data_list = []
    for file_name in files:
        # print(file_name)
        file_path = os.path.join(folder_path, file_name)
        # 读取文本文件
        data = pd.read_csv(file_path,  delimiter=',', skipinitialspace=True, header=0)
        # 清理列名中的空格
        data.columns = data.columns.str.strip()
        x_values = (data['MISES'][72616],data['MISES'][178908].astype(float),data['MISES'][128977].astype(float)) #提取指定的地方做传感器的输入
        x_list.append(x_values)
        # print(data.head())
        # 提取第二列数据
        second_column = data['MISES'].astype(float).values
        # 打印提取的第二列数据
        # print("Second column values:", second_column)
        data_list.append(second_column)
    
    data = torch.tensor(x_list).float()  
    label =  torch.tensor(np.array(data_list)).float()

    # 打印划分后的数据
    return data,label

def mean_squared_error(y_true, y_pred):
    return torch.mean((y_true - y_pred)**2)

In [3]:

current_directory = os.getcwd()
print("当前工作目录：", current_directory)



当前工作目录： /root/autodl-tmp


In [4]:
folder_path = r"/root/autodl-fs/ODB-E"
data,label = ReadTxtIntoDataset(folder_path)
data,label = data,label

In [6]:
print(data)

tensor([[ 40.1373,   1.6524, 139.7228],
        [ 56.3801,   2.2885, 192.1463],
        [103.6645,   4.1334, 349.2931],
        ...,
        [ 99.1270,   3.9372, 331.8474],
        [ 24.1407,   1.0321,  87.3307],
        [ 92.2133,   3.7398, 314.4116]])


In [7]:
class My_dataset(Dataset):
    def __init__(self,data,labels):
        self.data = data
        self.labels = labels
           
    def __getitem__(self, idx):
        return self.data[idx],self.labels[idx]
    
    def __len__(self):
        return len(self.data)

In [8]:
train_data, val_data, train_labels, val_labels = train_test_split(data, label, test_size=0.2, random_state=42)
train_dataset = My_dataset(train_data, train_labels)
val_dataset = My_dataset(val_data, val_labels)

In [9]:
class MLP(nn.Module):
    def __init__(self,input_size,hidden_size,output_size):
        super().__init__()
        self.mlp = nn.Sequential(
            nn.Linear(input_size,hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size,output_size),
        )
    def forward(self,x):
        x = self.mlp(x)
        return x

In [10]:
input = 3
hidden = 2000
out = 245293
model = MLP(input, hidden, out)
batch_size = 16
learning_rate = 0.01
num_epochs = 100

In [11]:
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
# 定义损失函数和优化器
criterion = nn.MSELoss()  # 均方误差损失函数
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
model = model.to(device)


cuda


In [10]:
model.load_state_dict(torch.load('MLP_E_model_weights.pth202434'))

<All keys matched successfully>

In [12]:
num_epochs = 5

In [37]:
print(val_data)
print(val_labels)

tensor([[4.6341e-03, 1.8623e-04, 1.5719e-02],
        [6.2350e-03, 2.5039e-04, 2.0962e-02],
        [4.5156e-03, 1.8640e-04, 1.5720e-02],
        ...,
        [5.2078e-03, 2.0881e-04, 1.7469e-02],
        [1.9952e-04, 3.9523e-06, 3.8282e-06],
        [4.5158e-03, 1.8692e-04, 1.5721e-02]])
tensor([[4.0177e-03, 3.5147e-03, 4.0200e-03,  ..., 4.1771e-03, 4.2291e-03,
         4.0865e-03],
        [5.3581e-03, 4.6873e-03, 5.3611e-03,  ..., 5.5679e-03, 5.6371e-03,
         5.4470e-03],
        [4.0169e-03, 3.5139e-03, 4.0191e-03,  ..., 4.1774e-03, 4.2294e-03,
         4.0867e-03],
        ...,
        [4.4652e-03, 3.9062e-03, 4.4678e-03,  ..., 4.6398e-03, 4.6974e-03,
         4.5391e-03],
        [1.5400e-06, 1.5908e-06, 1.5772e-06,  ..., 6.6784e-07, 6.3492e-07,
         7.3020e-07],
        [4.0171e-03, 3.5141e-03, 4.0193e-03,  ..., 4.1770e-03, 4.2290e-03,
         4.0864e-03]])


In [27]:
num_epochs = 100
total_steps = len(train_dataloader)
val_loss_optimize = 10
for epoch in range(num_epochs):
    model.train()
    for i, (inputs, labels) in enumerate(train_dataloader):
        # 前向传播
        # print(labels)
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        # print(outputs.size())
        loss = criterion(outputs, labels)

        # 反向传播和优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        torch.cuda.empty_cache()
        # 打印训练信息
        if (i + 1) % 10 == 0:
            print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{total_steps}], Loss: {loss.item():.8f}')
            
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_data, batch_labels in val_dataloader:
            # 前向传播
            outputs = model(batch_data.to(device))
#             print(f"outputs:{outputs}")
#             print(f"batch_labels:{batch_labels}")
            
#             loss = torch.mean(torch.mean(torch.abs(outputs - batch_labels.to(device))/batch_labels.to(device),axis = 0),axis = 1)
            loss = torch.mean(torch.mean(torch.abs(outputs - batch_labels.to(device))/batch_labels.to(device),axis = 1))
#             print(loss.item())
#             print(len(loss2))
            
            val_loss += loss.item()
#             print(val_loss)
#             print(len(val_dataloader))
            average_xiangdui_loss = val_loss / len(val_dataloader)
#             print(f"Val Loss: {average_xiangdui_loss:.8f}")
            # 计算误差
        
   
    
    print(f"Epoch {epoch+1}/{num_epochs}: Val Loss: {average_xiangdui_loss:.8f}")
#     if average_xiangdui_loss < val_loss_optimize:
#         torch.save(model.state_dict(), 'MLP_E_model_weights.pth202438')
#         val_loss_optimize = average_xiangdui_loss

Epoch [1/100], Step [10/127], Loss: 0.00614202
Epoch [1/100], Step [20/127], Loss: 0.01074389
Epoch [1/100], Step [30/127], Loss: 0.00691102
Epoch [1/100], Step [40/127], Loss: 0.01616484
Epoch [1/100], Step [50/127], Loss: 0.01307197
Epoch [1/100], Step [60/127], Loss: 0.01807889
Epoch [1/100], Step [70/127], Loss: 0.00999989
Epoch [1/100], Step [80/127], Loss: 0.00831980
Epoch [1/100], Step [90/127], Loss: 0.00620721
Epoch [1/100], Step [100/127], Loss: 0.00604198
Epoch [1/100], Step [110/127], Loss: 0.00864676
Epoch [1/100], Step [120/127], Loss: 0.00817076
Epoch 1/100: Val Loss: 0.20072010
Epoch [2/100], Step [10/127], Loss: 0.00851715
Epoch [2/100], Step [20/127], Loss: 0.00661816
Epoch [2/100], Step [30/127], Loss: 0.00866277
Epoch [2/100], Step [40/127], Loss: 0.00662009
Epoch [2/100], Step [50/127], Loss: 0.02223987
Epoch [2/100], Step [60/127], Loss: 0.01298276
Epoch [2/100], Step [70/127], Loss: 0.00953603
Epoch [2/100], Step [80/127], Loss: 0.01430227
Epoch [2/100], Step [90

In [ ]:
start = time.time()
input = val_data[5],label = val_labels[5]
output = model(input)
end = time.time()
print(f"time cost {end - start}s")
with open("time_cost_MLP.txt", "w") as file:
    file.write(str(end - start))  # 将变量转换为字符串并写入文件
    
error = torch.mean(torch.abs(output - label)/label,axis = 1)
np.save('MLP_error.npy', error)